! pip install fastparquet
! pip install --user annoy

## Fast Search -- Annoy (Approximate Nearest Neighbors Oh Yeah) 
https://github.com/spotify/annoy

In [1]:
import os
import pandas as pd
#
import numpy as np
from annoy import AnnoyIndex
import os
import time
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [141]:
# folder_dir = '/home/ds5749/ALS_10_0.01/'
folder_dir = '/home/ds5749/ALS100/'
# folder_dir = '/home/ds5749/ALS250/'
# folder_dir = '/home/ds5749/ALS500/'
concat = False
for path in os.listdir(os.path.join(folder_dir, 'itemFactors')):
    if 'part' in path:
        if not concat:
            item = pd.read_parquet(os.path.join(folder_dir, 'itemFactors', path))
            concat = True
        else:
            item = pd.concat([item, pd.read_parquet(os.path.join(folder_dir,'itemFactors', path))])
            
concat = False
for path in os.listdir(os.path.join(folder_dir, 'userFactors')):
    if 'part' in path:
        if not concat:
            user = pd.read_parquet(os.path.join(folder_dir, 'userFactors', path))
            concat = True
        else:
            user = pd.concat([item, pd.read_parquet(os.path.join(folder_dir,'userFactors', path))])
            
item = item.sort_values(by=['id'])
user = user.sort_values(by=['id'])
user = user[user.id.isin(users)]

In [142]:
val = pd.read_csv('/home/ds5749/data/val_large.csv')
users = user.id.unique()[0:100]
val = val[val.userId.isin(users)]
train = pd.read_csv('/home/ds5749/data/train_large.csv')
train = val[val.userId.isin(users)]
label = pd.concat([val, train])

In [143]:
rating = []
for ID in users:
    lst = label[label.userId == ID][['movieId', 'rating']].sort_values(by = ['rating'], ascending = False)
    rating.append(lst.movieId.values)

In [144]:
class Inference:
    def __init__(self, item_vec, movie_id):
        self.data = item_vec
        self.movie_id = movie_id
    def Create_Approximate_Nearest_Neighbors(self, item, n_trees=40):
        f = len(item.features.iloc[0])
        self.Approximate_Nearest_Neighbors = AnnoyIndex(f, metric='dot')
        for i in item.id:
            self.Approximate_Nearest_Neighbors.add_item(i, item[item.id==i].features.values[0])
        self.Approximate_Nearest_Neighbors.build(n_trees, n_jobs=-1)
      
    def search(self, queries,algo, k):
        times = []
        results = []
        for i in tqdm(range(len(queries))):
            interval, res = self.single_query(queries[i], algo, k)
            times.append(interval)
            results.append(res)
        mean_time = sum(times) / len(times)
        print(algo)
        print('Average Query Time: %.8f' % mean_time)
        return mean_time, results    
        
    def single_query(self, query, algo, k):
        start = time.time()
        if type(query) is not np.ndarray:
            query = np.array(query)
        res = self.similarity(query, algo, k)
        interval = time.time() - start
        return interval, res
    
    def similarity(self, query, algo, k):
        if algo == "brute_force":
            res = np.argsort(-self.data.dot(query))[:k]
            lst = []
            for i in res:
                lst.append(self.movie_id[i])
            return lst
                
        elif algo == "Approximate_Nearest_Neighbors":
            return self.Approximate_Nearest_Neighbors.get_nns_by_vector(query, n=k, search_k=-1, include_distances=False)
        
def precision(y_pred, y_true):
    hit = [1 if y_pred[i] in y_true else 0 for i in range(len(y_pred))]
    return sum(hit)/len(hit)

In [145]:
movie_vec = []
for i in item.id:
    movie_vec.append(np.array(item[item.id==i].features.values[0]))
movie_id =  [i for i in item.id]
movie_vec = np.vstack(movie_vec)
user_vec = []
for i in user.id[0:100].unique():
    user_vec.append(user[user.id==i].features.values[0])

In [146]:
inference = Inference(movie_vec, movie_id)
mean_time, result = inference.search(user_vec, 'brute_force', k=100)
p = 0
for i in range(len(result)):
    p += precision(result[i], rating[i])
print("Precision:", p/len(result))

<ipython-input-144-0ad35759326b>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(queries))):


  0%|          | 0/8 [00:00<?, ?it/s]

brute_force
Average Query Time: 0.00409326
Precision: 0.00125


In [150]:
inference.Create_Approximate_Nearest_Neighbors(item, n_trees=8)
mean_time, results = inference.search(user_vec, 'Approximate_Nearest_Neighbors', k=100)
p = 0
for i in range(len(results)):
    p += precision(results[i], rating[i])
print("Precision:", p/len(results))

<ipython-input-144-0ad35759326b>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(queries))):


  0%|          | 0/8 [00:00<?, ?it/s]

Approximate_Nearest_Neighbors
Average Query Time: 0.00010315
Precision: 0.0025
